# Tarea 7: Muestreo estratificado

Solución de los ejercicios de la sección `Intervalos de confianza y remuestreo` del libro del curso [Fundamentos Estadísticos](https://tereom.github.io/fundamentos-2022/intervalos-de-confianza-y-remuestreo.html)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadr

In [2]:
path_universal = "C:\\Users\\Miguel\\Documents\\Github\\itam-mcd"
os.listdir(path_universal)

['.git',
 '.gitignore',
 'aprendizaje-automatico',
 'estadistica-computacional',
 'fundamentos-estadisticos',
 'mineria-analisis-datos',
 'optimizacion',
 'README.md']

In [3]:
path_asignatura = "fundamentos-estadisticos"
path_asignatura = os.path.join(path_universal, path_asignatura)
os.listdir(path_asignatura)

['datos', 'libros', 'README.md']

## Conteo rápido

En México, las elecciones tienen lugar un domingo, los resultados oficiales del proceso se presentan a la población una semana después. A fin de evitar proclamaciones de victoria injustificadas durante ese periodo, el INE organiza un conteo rápido. El conteo rápido es un procedimiento para estimar, a partir de una muestra aleatoria de casillas, el porcentaje de votos a favor de los candidatos en la elección. 

En este ejercicio deberás crear intervalos de confianza para la proporción de votos que recibió cada candidato en las elecciones de 2006. La inferencia se hará a partir de una muestra de las casillas similar a la que se utilizó para el conteo rápido de 2006.

El diseño utilizado es **muestreo estratificado simple**, lo que quiere decir que:
1) Se particionan las casillas de la población en estratos (cada casilla pertenece a exactamente un estrato), y 
2) Dentro de cada estrato se usa *muestreo aleatorio* para seleccionar las casillas que estarán en la muestra. 

En este ejercicio (similar al conteo rápido de 2006):
* Se seleccionó una muestra de $7,200$ casillas
* La muestra se repartió a lo largo de 300 estratos. 
* La tabla `strata_sample_2006` contiene en la columna $N$ el número total de casillas en el estrato, y en $n$ el número de casillas que se seleccionaron en la muestra, para cada estrato.

In [4]:
#Se obtiene dataset original
file_path = "datos\\strata_sample_2006.rda"
strata_sample_2006 = pyreadr.read_r(os.path.join(path_asignatura, file_path))
strata_sample_2006 = strata_sample_2006['strata_sample_2006']
strata_sample_2006['factor'] = strata_sample_2006['N'] / strata_sample_2006['n']

strata_sample_2006

,stratum,n,N,factor
0,1.0,20,369,18.450000
1,2.0,23,420,18.260870
2,3.0,24,440,18.333333
3,4.0,31,570,18.387097
4,5.0,29,528,18.206897
...,...,...,...,...
295,296.0,23,425,18.478261
296,297.0,25,452,18.080000
297,298.0,26,472,18.153846
298,299.0,29,531,18.310345


In [5]:
strata_sample_2006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   stratum  300 non-null    float64
 1   n        300 non-null    int32  
 2   N        300 non-null    int32  
 3   factor   300 non-null    float64
dtypes: float64(2), int32(2)
memory usage: 7.2 KB


* La tabla `sample_2006` en el paquete `estcomp` contiene para cada casilla:
    + el estrato al que pertenece: `stratum`
    + el número de votos que recibió cada partido/coalición: `pan`, `pri_pvem`, `panal`, `prd_pt_convergencia`, `psd` y la columna `otros` indica el número de votos nulos o por candidatos no registrados.
    + el total de votos registrado en la casilla: `total`.

In [6]:
#Se obtiene dataset original
file_path = "datos\\sample_2006.rda"
sample_2006 = pyreadr.read_r(os.path.join(path_asignatura, file_path))
sample_2006 = sample_2006['sample_2006']

#Se une dataset con factor de representación
sample_2006 = pd.merge(left=sample_2006, right=strata_sample_2006, on='stratum')

sample_2006.head()

,polling_id,stratum,edo_id,rural,pri_pvem,pan,panal,prd_pt_conv,psd,otros,total,n,N,factor
0,74593,106.0,16,1.0,47,40,0,40,0,9,136,26,478,18.384615
1,74390,106.0,16,0.0,172,41,2,41,1,4,261,26,478,18.384615
2,74757,106.0,16,0.0,32,136,2,127,3,12,312,26,478,18.384615
3,74693,106.0,16,0.0,28,54,2,13,1,5,103,26,478,18.384615
4,74779,106.0,16,0.0,32,93,2,129,10,14,280,26,478,18.384615


Una de las metodolgías de estimación que se usa en el conteo rápido es **estimador de razón**, y se contruyen intervalos de 95% de confianza usando el método normal con error estándar bootstrap. En este ejercicio debes construir intervalos usando este procedimiento.
<br>Para cada candidato:
1) Calcula el estimador de razón combinado, para muestreo estratificado la fórmula es:
$$\hat{p}=\frac{\sum_h \frac{N_h}{n_h} \sum_i Y_{hi}}{\sum_h \frac{N_h}{n_h} \sum_i X_{hi}}$$
<br>Donde:
* $\hat{p}$ es la estimación de la proporción de votos que recibió el candidato en la elección.
* $Y_{hi}$ es el número total de votos que recibió el candidato en la $i$-ésima casillas, que pertence al $h$-ésimo estrato.
* $X_{hi}$ es el número total de votos en la $i$-ésima casilla, que pertence al $h$-ésimo estrato.
* $N_h$ es el número total de casillas en el $h$-ésimo estrato.
* $n_h$ es el número de casillas del $h$-ésimo estrato que se seleccionaron en la muestra.

2) Utiliza **bootstrap** para calcular el error estándar, y reporta tu estimación del error.
    + Genera 1000 muestras bootstrap.
    + Recuerda que las muestras bootstrap tienen que tomar en cuenta la metodología que se utilizó en la selección de la muestra original, en este caso, lo que implica es que debes tomar una muestra aleatoria independiente dentro de cada estrato.
3) Construye un intervalo del 95% de confianza utilizando el método normal. Revisa si el supuesto de normalidad es razonable.
4) Reporta tus intervalos en una tabla. 

**Paso 1**: Se estiman las proporciones para todos los partidos

In [7]:
#Diccionario para almacenar información de cada partido
partidos = {}
for partido in ['pri_pvem', 'pan', 'panal', 'prd_pt_conv', 'psd', 'otros']:
    partidos[partido] = {}

In [8]:
#Estimador de razón para cada partido
for partido in partidos.keys():
    #Se expanden votos con factor de representación
    votos_partido = (sample_2006[partido] * sample_2006['factor']).sum()
    votos_totales = (sample_2006['total'] * sample_2006['factor']).sum()
    est_razon = votos_partido / votos_totales
    
    #Se almacena estimador muestral en diccionario
    partidos[partido]['prop'] = est_razon
    
    #Se genera llave para almacenar información de bootstrap
    partidos[partido]['boots_prop'] = np.nan
    partidos[partido]['std_err'] = np.nan
    partidos[partido]['left_int'] = np.nan
    partidos[partido]['right_int'] = np.nan
    partidos[partido]['boots_dist'] = []
    
partidos

{'pri_pvem': {'prop': 0.22324286439556812,
  'boots_prop': nan,
  'std_err': nan,
  'left_int': nan,
  'right_int': nan,
  'boots_dist': []},
 'pan': {'prop': 0.3591955774857583,
  'boots_prop': nan,
  'std_err': nan,
  'left_int': nan,
  'right_int': nan,
  'boots_dist': []},
 'panal': {'prop': 0.009521229237363519,
  'boots_prop': nan,
  'std_err': nan,
  'left_int': nan,
  'right_int': nan,
  'boots_dist': []},
 'prd_pt_conv': {'prop': 0.3519762177808759,
  'boots_prop': nan,
  'std_err': nan,
  'left_int': nan,
  'right_int': nan,
  'boots_dist': []},
 'psd': {'prop': 0.027066319097616177,
  'boots_prop': nan,
  'std_err': nan,
  'left_int': nan,
  'right_int': nan,
  'boots_dist': []},
 'otros': {'prop': 0.02899779200281792,
  'boots_prop': nan,
  'std_err': nan,
  'left_int': nan,
  'right_int': nan,
  'boots_dist': []}}

**Paso 2**: Utiliza **bootstrap** para calcular el error estándar, y reporta tu estimación del error.

**Primero** generamos muestras aleatorias de los índicies de `sample_2006` con reemplazo para cada uno de los estratos en `stratum`, que sean del mismo tamaño que las muestras originales de estrato.

In [9]:
#Se obtienen etiquetas de estrato
est_id = sample_2006['stratum'].unique()
est_id.sort()

#Función para generar un vector de índices muestreados aleatoriamente con reemplazo
def muestreo_estratros():
    #Vector vacío para almacenar índices de muestras estratificadas
    estratos = np.array([])
    
    for idx in est_id:
        #Filtro por estrato
        estrato_df = sample_2006['stratum'].loc[sample_2006['stratum'] == idx]

        #Muestreo de índices con reemplazo
        rand_samp = np.random.choice(a=estrato_df.index, size=len(estrato_df), replace=True)

        estratos = np.append(estratos, rand_samp)
    
    return estratos.astype(int)

muestreo_estratros()[:20]

array([864, 859, 870, 867, 877, 858, 858, 875, 866, 870, 864, 865, 864,
       858, 865, 872, 871, 858, 876, 869])

**Después** se obtienen los factores de proporción por partido para cada muestra estratificada, se almacena en el diccionario de partido con las claves `boots_dist` y `prop_boots` para $1,000$ muestras bootstrap

In [10]:
%%time
for _ in range(1_000):
    #Estimador de razón para cada partido
    for partido in partidos.keys():
        bootstrap_sample = sample_2006[[partido, 'total', 'factor']].loc[muestreo_estratros()].to_numpy()

        #Se expanden votos con factor de representación
        votos_partido = (bootstrap_sample[:,0] * bootstrap_sample[:,2]).sum()
        votos_totales = (bootstrap_sample[:,1] * bootstrap_sample[:,2]).sum()
        est_razon = votos_partido / votos_totales

        #Se almacena estimador muestral en diccionario
        partidos[partido]['boots_dist'].append(est_razon)

Wall time: 5min 13s


**Paso 3**: Construyendo intervalo de confianza

In [11]:
for partido in partidos.keys():
    #Calculando estimador bootsrap
    partidos[partido]['boots_prop'] = np.array(partidos[partido]['boots_dist']).mean()

    #Calculando error estándar
    partidos[partido]['std_err'] = np.array(partidos[partido]['boots_dist']).std()
    
    #Límites del intervalo
    partidos[partido]['left_int'] = partidos[partido]['boots_prop'] - partidos[partido]['std_err']
    partidos[partido]['right_int'] = partidos[partido]['boots_prop'] + partidos[partido]['std_err']

In [12]:
specs = ['prop', 'boots_prop', 'std_err', 'left_int', 'right_int']
resumen = pd.DataFrame(index=partidos.keys(), columns=specs)

for partido in resumen.index:
    for spec in specs:
        resumen.loc[partido, spec] = partidos[partido][spec]

resumen

,prop,boots_prop,std_err,left_int,right_int
pri_pvem,0.223243,0.223236,0.001022,0.222213,0.224258
pan,0.359196,0.359189,0.001304,0.357885,0.360493
panal,0.009521,0.009517,0.000126,0.009391,0.009643
prd_pt_conv,0.351976,0.351923,0.001171,0.350752,0.353094
psd,0.027066,0.02707,0.000177,0.026893,0.027248
otros,0.028998,0.029002,0.000338,0.028665,0.02934
